# NIH Chest X-Ray Dataset: Dimensionality Reduction

## Load Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as transforms
from torch.utils.data import Subset
from torchvision.utils import make_grid
from torchxrayvision.datasets import NIH_Dataset, XRayCenterCrop, XRayResizer

from drift_detection.drift_detector import DCTester, Reductor, TSTester

## Load Dataset

In [ ]:
# paths for NIH dataset
IMAGE_PATH = "~/NIHCXR/images"
CSV_PATH = "~/NIHCXR/Data_Entry_2017.csv"

# load NIH dataset
dataset = NIH_Dataset(
    IMAGE_PATH,
    CSV_PATH,
    views=["AP", "PA"],
    unique_patients=False,
    transform=transforms.Compose([XRayCenterCrop(), XRayResizer(224, engine="cv2")]),
)

# grab random subset of 400 image indices from dataset
indices = np.random.randint(len(dataset), size=400)
dataset = Subset(dataset, indices)

## Sample Images

In [ ]:
imgs = torch.concat([torch.tensor(img["img"]) for img in dataset]).unsqueeze(1)

plt.figure(figsize=(20, 10))
plt.imshow(
    make_grid(imgs[np.random.randint(400, size=16)], normalize=True).permute(1, 2, 0),
    cmap="gray",
)

## TorchXRayVision Trained Autoencoder

    Reduce Dimensionality of images by using the representations extracted from the bottleneck of the 
    TorchXRayVision trained autoencoder.

```X_input``` shape: (400, 1, 224, 224)

In [ ]:
from drift_detection.drift_detector import Reductor

reductor = Reductor("TAE_txrv_CNN")
X_transformed, y = reductor.transform(dataset, num_workers=4)

In [ ]:
print("X_transformed shape: ", X_transformed.shape, "\tlabels shape: ", y.shape)

## TorchXRayVision Classifier (BBSDs)
    Reduce dimensionality of images with the TorchXRayVision Classifier; returns the softmax probability 
    distribution of multi-class prediction of 18 pathologies.

In [ ]:
reductor = Reductor("BBSDs_txrv_CNN")
X_transformed, y = reductor.transform(dataset, num_workers=4)

In [ ]:
print("X_transformed shape: ", X_transformed.shape, "\tlabels shape: ", y.shape)

The intersection of the model predictions and the NIH dataset labels yield 10 overlapping pathologies:

In [ ]:
np.intersect1d(reductor.model.pathologies, dataset.dataset.pathologies)